In [37]:
import keras
import pandas as pd
import matplotlib.pyplot as plt 
from datetime import datetime  
from nltk.tokenize import word_tokenize  
import seaborn as sns
import numpy as np
import re

import pickle

from sklearn import preprocessing
from sklearn.preprocessing import LabelEncoder, LabelBinarizer, OneHotEncoder

from sklearn.metrics import accuracy_score,roc_auc_score, f1_score, recall_score
from sklearn.metrics import  roc_curve, confusion_matrix, precision_score



from sklearn.model_selection import train_test_split
from sklearn.model_selection import GridSearchCV, StratifiedKFold

from sklearn.naive_bayes import GaussianNB
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegression


from nltk.tokenize import TweetTokenizer
from keras.preprocessing.text import Tokenizer
# function to make n-grams
from nltk.util import ngrams 
from nltk import everygrams
from keras.preprocessing.sequence import pad_sequences

from sklearn.feature_extraction.text import TfidfVectorizer

# tfidf = TfidfVectorizer(vocabulary = myvocabulary, stop_words = 'english')
# tfs = tfidf.fit_transform(corpus.values())

from keras import layers
from keras import optimizers
from keras import models
from keras import losses
from keras import metrics

from gensim.corpora import Dictionary
import itertools
from collections import defaultdict
from wordcloud import WordCloud
import warnings
warnings.filterwarnings('ignore') 
%matplotlib inline

In [38]:
tk = Tokenizer()
def tokenize_tweet(s):
    """
    Tokenize each text into a list of words 
    """
    tokens = tk(s)
    return [w for w in tokens]

In [39]:
df = pd.read_csv('../data/processed/data_for_modelling.csv')

# TF-IDF Vectorization

In [40]:
# instantiate the tf-idf object
tfidf_vectors = TfidfVectorizer(max_df=0.90, min_df=2, max_features=9000, 
                                stop_words='english',
                                ngram_range=(1, 3))



In [41]:
tfidf_only_fit = tfidf_vectors.fit(df['tweet_without_stopwords_and_2'])

In [42]:
# pickling only the fit

pickle_out_only_fit = open('tfidf_pickle_fit', 'wb')
pickle.dump(tfidf_only_fit, pickle_out_only_fit)

In [43]:
tfidf = tfidf_vectors.fit_transform(df['tweet_without_stopwords_and_2'])

In [44]:
# pickling the matrix 

pickle_out_matrix = open('tfidf_pickle', 'wb')
pickle.dump(tfidf, pickle_out_matrix)

In [45]:
df_vector = pd.DataFrame(tfidf.todense(),columns = tfidf_vectors.get_feature_names())

# Splitting into Training/Validation/Test

In [46]:
target = df['neg_label']


In [47]:
x_train, x_test, y_train , y_test = train_test_split(df_vector, target, 
                                                     test_size =.2, random_state=101 )

In [48]:
print(f'The shape of my training set is {x_train.shape}')
print(f'The shape of my training target is {y_train.shape}')
print(f'The shape of my test set is {x_test.shape}')
print(f'The shape of my test target is {y_test.shape}')

The shape of my training set is (33593, 9000)
The shape of my training target is (33593,)
The shape of my test set is (8399, 9000)
The shape of my test target is (8399,)


In [49]:
X_train, x_val, Y_train , y_val = train_test_split(x_train,y_train, 
                                                     test_size =.2, random_state=101 )

In [50]:
print(f'The shape of my final training set is {X_train.shape}')
print(f'The shape of my final training target is {Y_train.shape}')
print(f'The shape of my validation set is {x_val.shape}')
print(f'The shape of my validation target is {y_val.shape}')

The shape of my final training set is (26874, 9000)
The shape of my final training target is (26874,)
The shape of my validation set is (6719, 9000)
The shape of my validation target is (6719,)


# Logistic Regression Baseline

In [51]:
logmodel = LogisticRegression()

In [52]:
logmodel.fit(X_train, Y_train)

LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
                   intercept_scaling=1, l1_ratio=None, max_iter=100,
                   multi_class='warn', n_jobs=None, penalty='l2',
                   random_state=None, solver='warn', tol=0.0001, verbose=0,
                   warm_start=False)

In [53]:
# testing on validation
y_hat_log = logmodel.predict(x_test)





In [54]:
f1_score(y_test, y_hat_log) # calculating f1 score

0.9804890096320079

In [55]:


confusion_matrix(y_test, y_hat_log)

array([[4271,   36],
       [ 122, 3970]])

In [56]:
tn, fp, fn, tp = confusion_matrix(y_test, y_hat_log).ravel()
tn, fp, fn, tp

(4271, 36, 122, 3970)

In [57]:
y_hat_proba = logmodel.predict_proba(x_test)

In [58]:
y_hat_proba[:10]

array([[0.93023185, 0.06976815],
       [0.86738193, 0.13261807],
       [0.01654884, 0.98345116],
       [0.9510001 , 0.0489999 ],
       [0.06449726, 0.93550274],
       [0.86595032, 0.13404968],
       [0.10116755, 0.89883245],
       [0.12675282, 0.87324718],
       [0.96427155, 0.03572845],
       [0.96041194, 0.03958806]])

In [59]:
y_test

21489    0
14714    0
12192    1
31163    0
6053     1
        ..
28287    1
21944    0
26081    0
32660    1
2474     1
Name: neg_label, Length: 8399, dtype: int64

In [60]:
Y_hat_proba = y_hat_proba[:,1]

In [61]:
Y_hat_proba[:10]

array([0.06976815, 0.13261807, 0.98345116, 0.0489999 , 0.93550274,
       0.13404968, 0.89883245, 0.87324718, 0.03572845, 0.03958806])

In [62]:
class_neg = Y_hat_proba > .7

In [63]:
class_neg

array([False, False,  True, ..., False,  True,  True])

# Pickling the final winning model

In [64]:
logmodel

LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
                   intercept_scaling=1, l1_ratio=None, max_iter=100,
                   multi_class='warn', n_jobs=None, penalty='l2',
                   random_state=None, solver='warn', tol=0.0001, verbose=0,
                   warm_start=False)

In [65]:
pickle_out = open('pickled_logistic_1', 'wb')
pickle.dump(logmodel, pickle_out)

# Convolutional Neural Network

In [184]:
df

,tweet_low,handle_count,no_handle_no_special_no_sin_ash,tweet_length,character_count,lemmatized,lemmatized_1,lemmatized_final,tweet_without_stopwords,tweet_without_stopwords_and_2,neg_label
0,13 days to go #gettingthere,0,days to go #gettingthere,30,21,day to go # gettingthere,day to go #gettingthere,day to go # gettingthere,day go # gettingthere,day gettingthere,0
1,@anggxo get off my twitter fag,1,get off my twitter fag,23,18,get off my twitter fag,get off my twitter fag,get off my twitter fag,get twitter fag,get twitter fag,1
2,these hoes got more bodies than a cemetery&#12...,0,these hoes got more bodies than a cemetery ...,58,35,these hoe get more body than a cemetery,these hoe get more body than a cemetery,these hoe get more body than a cemetery,hoe get body cemetery,hoe get body cemetery,1
3,a friend just told me she's afraid to go to dc...,1,a friend just told me she's afraid to go to dc...,124,95,a friend just tell me she 's afraid to go to d...,a friend just tell me she 's afraid to go to d...,a friend just tell me she's afraid to go to dc...,friend tell 's afraid go dc rally amp attack #...,friend tell afraid rally amp attack berniebros...,0
4,i've noticed a lot of #icontf16 presentations ...,1,i've noticed a lot of #icontf presentations ...,117,95,i 've notice a lot of # icontf presentation me...,i 've notice a lot of #icontf presentation men...,i've notice a lot of # icontf presentation men...,'ve notice lot # icontf presentation mention h...,'ve notice lot icontf presentation mention hap...,0
...,...,...,...,...,...,...,...,...,...,...,...
41987,@_elenaraquel_ its swag bitch aha,1,its swag bitch aha,19,15,its swag bitch aha,its swag bitch aha,its swag bitch aha,swag bitch aha,swag bitch aha,1
41988,when quay collab with @user says sold out!!!ð...,1,when quay collab with says sold out ...,121,54,when quay collab with say sell out # noooo # w...,when quay collab with say sell out #noooo #why...,when quay collab with say sell out # noooo # w...,quay collab say sell # noooo # whyyyy # loveyo...,quay collab say sell noooo whyyyy loveyoudesi,0
41989,rt @_groovymovie: &#8220;@shane_a1: hmu talmbo...,2,rt hmu talmbout match but when i pull ...,114,73,rt hmu talmbout match but when i pull up its o...,rt hmu talmbout match but when i pull up its o...,rt hmu talmbout match but when i pull up its o...,rt hmu talmbout match pull niccas wit ya smfh ...,hmu talmbout match pull niccas wit smfh shit l...,1
41990,rt @obey_jrock__: this is a true ride or die b...,1,rt this is a true ride or die bitch,45,27,rt this be a true ride or die bitch,rt this be a true ride or die bitch,rt this be a true ride or die bitch,rt true ride die bitch,true ride die bitch,1


In [185]:
# Some variables that will be used throughout the modeling process

epochs = 10  
batch_size = 64  
sequence_len = 23  # Maximum number of words in a sequence
embedding_dimensions = 100  # Number of dimensions of the trainable embedding layer

In [186]:
tknz = TweetTokenizer()

def tokenize_tweet(s):
    """
    Tokenize each text into a list of words removing the ashtags
    """
    tokens = tknz.tokenize(s)
    return [w for w in tokens]

In [187]:
df['tokenized'] = df['tweet_without_stopwords_and_2'].apply(tokenize_tweet)

In [188]:
df['tokenized']

0                                      [day, gettingthere]
1                                      [get, twitter, fag]
2                               [hoe, get, body, cemetery]
3        [friend, tell, afraid, rally, amp, attack, ber...
4        [', ve, notice, lot, icontf, presentation, men...
                               ...                        
41987                                   [swag, bitch, aha]
41988    [quay, collab, say, sell, noooo, whyyyy, lovey...
41989    [hmu, talmbout, match, pull, niccas, wit, smfh...
41990                             [true, ride, die, bitch]
41991    [get, deal, low, know, hoe, lonely, fuck, cuff...
Name: tokenized, Length: 41992, dtype: object

In [189]:

X_train, X_test, y_train, y_test = train_test_split(df['tokenized'], df['neg_label'], test_size=0.2, random_state=49)

In [190]:
tk = Tokenizer(num_words=20000, # The 20000 most important words for our vocabulary
               filters='!"#$%&()*+,-./:;<=>?@[\]^_`{"}~\t\n', # Regex that will prune off non alphabetical characters
               lower=True, # Lowercase the text data for consistency
               split=" ")
tk.fit_on_texts(X_train) # Fit on our training set
X_train_seq = tk.texts_to_sequences(X_train) # Indexed sequences based on fitted tokenizer
X_test_seq = tk.texts_to_sequences(X_test)

In [191]:
def max_seq_length (sequence):
    length = []
    
    for i in range(0, len(sequence)):
        length.append(len(sequence[i]))
    return max(length)

    
   
  




The length of max_sequence is 23

In [192]:
max_seq_length(X_train_seq)

23

In [193]:
max_length = 23

In [194]:
# # 19000 seems to be range of vocabulary
# for i in range(0, len(X_train_seq)-1):
#     highest_words = []
#     highest_words.append(max(X_train_seq[i]))
#     print(highest_words)
        
     

In [195]:
number_of_words = 20000

In [196]:
X_train_seq_pad = pad_sequences(X_train_seq, maxlen=max_length) # Padding the sequences so that they are all the same length
X_test_seq_pad = pad_sequences(X_test_seq, maxlen=max_length)

In [197]:
# checking for one length
X_train_seq_pad[0]

array([   0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
          0,    0,    2,  460,    4,   17,    3,  122,  124, 1390,   16,
          1], dtype=int32)

In [198]:

X_train_emb, X_valid_emb, y_train_emb, y_valid_emb = train_test_split(X_train_seq_pad, y_train, test_size=0.2, random_state=42)

In [199]:
# Some variables that will be used throughout the modeling process

epochs = 10  
batch_size = 64  
embedding_dimensions = 100  # Number of dimensions of the trainable embedding layer

In [200]:

emb_model = models.Sequential()
emb_model.add(layers.Embedding(number_of_words, embedding_dimensions, input_length=max_length))
emb_model.add(layers.Conv1D(32,
                            5,
                            activation='relu',
                            input_shape=(100,1)))
emb_model.add(layers.GlobalMaxPooling1D())
emb_model.add(layers.Dropout(0.2))
emb_model.add(layers.Dense(64,activation='relu',))
emb_model.add(layers.Dropout(0.2))
emb_model.add(layers.Dense(1, activation='sigmoid'))
emb_model.compile(optimizer='adam',
                  loss='binary_crossentropy',
                  metrics=['accuracy'])
emb_history = emb_model.fit(X_train_emb,
                            y_train_emb,
                            epochs=epochs,
                            batch_size=batch_size,
                            validation_data=(X_valid_emb, y_valid_emb))

Train on 26874 samples, validate on 6719 samples
Epoch 1/10
26874/26874 [==============================] - 17s 623us/step - loss: 0.1723 - accuracy: 0.9324 - val_loss: 0.0578 - val_accuracy: 0.9836
Epoch 2/10
26874/26874 [==============================] - 17s 620us/step - loss: 0.0326 - accuracy: 0.9915 - val_loss: 0.0571 - val_accuracy: 0.9839
Epoch 3/10
26874/26874 [==============================] - 16s 610us/step - loss: 0.0178 - accuracy: 0.9961 - val_loss: 0.0662 - val_accuracy: 0.9829
Epoch 4/10
26874/26874 [==============================] - 16s 594us/step - loss: 0.0123 - accuracy: 0.9970 - val_loss: 0.0743 - val_accuracy: 0.9814
Epoch 5/10
26874/26874 [==============================] - 16s 601us/step - loss: 0.0081 - accuracy: 0.9980 - val_loss: 0.0841 - val_accuracy: 0.9807
Epoch 6/10
26874/26874 [==============================] - 17s 618us/step - loss: 0.0053 - accuracy: 0.9988 - val_loss: 0.0988 - val_accuracy: 0.9805
Epoch 7/10
26874/26874 [==============================] -

In [201]:
emb_model.evaluate(X_test_seq_pad, y_test)

8399/8399 [==============================] - 0s 59us/step


[0.1076549892671105, 0.9792832732200623]

In [202]:
probabilities_for_test = emb_model.predict_proba(X_test_seq_pad)

In [203]:
probabilities_for_test[:10]

array([[8.6107068e-03],
       [1.0000000e+00],
       [8.2513674e-10],
       [2.0167288e-06],
       [9.9986053e-01],
       [5.8676579e-07],
       [9.9996185e-01],
       [1.0378705e-02],
       [9.9929070e-01],
       [9.9817097e-01]], dtype=float32)

In [204]:
probabilities_for_test.max()

1.0

# Pickling the CNN

In [157]:
pickle_cnn = open('pickle_cnn', 'wb')
pickle.dump(emb_model,pickle_cnn)

# Naive Bayes

In [30]:
from sklearn.feature_extraction.text import CountVectorizer
cv = CountVectorizer(max_features=9000)

X = cv.fit_transform(df['tweet_without_stopwords_and_2']).toarray()


from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, target, test_size = 0.2, random_state = 10)

from sklearn.naive_bayes import GaussianNB
classifier = GaussianNB()
classifier.fit(X_train, y_train)



GaussianNB(priors=None, var_smoothing=1e-09)

Pickling the count vectorizer

In [31]:
pickle_out_cv = open('CVBayes', 'wb')
pickle.dump(cv, pickle_out_cv)

Pickling the Naives Model

In [32]:
pickle_out_bayes = open('BayesModel', 'wb')
pickle.dump (classifier, pickle_out_bayes )

In [33]:
y_pred = classifier.predict(X_test)

from sklearn.metrics import confusion_matrix
cm = confusion_matrix(y_test, y_pred)

In [34]:
cm

array([[2402, 1852],
       [ 195, 3950]])